<h1 style="color:#333333; text-align:center; line-height: 0;">Reinforcement Learning | Assignment 1</h1>

<br/><br/>

This notebook covers two classical approaches in RL: **value iteration** and **policy iteration**.

Before you start, please install OpenAI gym for Python [here](https://gym.openai.com/docs/).

Complete the code snippets given in the Section 3: there are 4 places to insert your code, one string field to enter the algorithm name and string fields for your first and last name. The latter are needed to automatically save the results of the algorithms deployment in .json file. After you did that, please upload the notebook (.ipynb) and .json via https://forms.gle/yJepv3CzfwMRnQULA.

* Problem 1.1 - Value Iteration (15 points)
* Problem 1.2 - Policy Iteration (15 points)
* Problem 1.3 - FrozenLake8x8 (2 points)

***

<h2 style="color:#A7BD3F;">Section 1: Theory recap</h2>

$x$ - state from state space $\mathbb{X}$

$u$ - action from action space $\mathbb{U}$

$R$ - reward function as a random variable ($\rho$ as a deterministic function)

$\kappa$ - policy (in deterministic case)

$\vartheta$ - policy parameters from parameter space $\Pi$

Discrete time deterministic system (DT):

$x_{k+1} = f(x_k, u_k)$

$u_k = \kappa(x_k)$

The core problem that is being solved in RL is teaching the agent to behave *optimally* in a specific environment.

The numerical definition of what is *optimal* is given by the **objective function** $J$, which is typically maximized in the context of MDPs (Markov Decision Processes):

$X_{k+1} \sim P_{X}\left(x_{k+1} \mid x_{u}, u_{n}\right)$

$U_{k} \sim P_{U}^{\vartheta}\left(u_{k} \mid x_{k}\right)$

The next state and next control are sampled from the respective probability distributions. $P_{X}, P_{U}^{\vartheta}$ are probability distribution (or density if $\mathbb{X}$, $\mathbb{U}$ are continuous) functions (PDF). Policy is represented here by a PDF with parameters $\vartheta$, but may be a definite function (Markov policy) $ = \kappa(X_{k})$

The goal then is to teach the agent to maximize the *total expected reward* it earns over some time horizon (theoretically, it is an infinite time horizon) by selecting the best action as dictated by the value function. The agent learns to maximize rewards as it transitions from state to state, taking actions in each state.

Consider the following $\infty$-horizon optimal control problem for an MDP:
$$
J(x) = \max _{\vartheta} \mathbb{E}\left[\sum_{k=0}^{\infty} \gamma^{k} \rho\left(X_{k}, \kappa\left(X_{k}\right)\right) \mid X_{0}=x\right]
$$

The value function is defined in the following way:

$V(x) = \underset{\vartheta}{\max} \ J^{\vartheta}(x)$

Let us use the denotation $X_{+}^{u} \sim P_{X}(x \mid x, u)$ for the next state.

***

<h2 style="color:#A7BD3F;">Section 2: OpenAI FrozenLake environment</h2>

For this homework we will be exploring agent training in the grid world environment called *FrozenLake*. Read more about it [here](https://gym.openai.com/envs/FrozenLake-v0/). To summarize:
* FrozenLake is a 2D grid world
* There are 2 variants of the environment: 4x4 and 8x8. We will try both.
* There are 4 types of grid cells (S, F, H, G). *S* is the starting point, *G* is the goal, *F* is a frozen surface and *H* is a hole.
* If an agent steps onto a slippery surface, it may slip and not end up in the next desired state for which it took an action (think, transition probabilities!).
* The rewards are sparse: the agent receives a reward of 1 when reaching the goal and 0 otherwise. If the agent falls into a hole, the episode is over.
* Note: we will not be using the slippery version of the environment for simplicity.

Familiarize yourself with the code below and run it.

In [1]:
import gym
import numpy as np
import collections
import sys
from tqdm import tqdm
from IPython.display import clear_output
import time

In [2]:
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print(observation)
        if done:
            #print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[ 0.00626441 -0.00374813 -0.01639127  0.00662839]
[ 0.00618945  0.19160502 -0.01625871 -0.2911808 ]
[ 0.00618945  0.19160502 -0.01625871 -0.2911808 ]
[ 0.01002155 -0.00328138 -0.02208232 -0.00366967]
[ 0.01002155 -0.00328138 -0.02208232 -0.00366967]
[ 0.00995592 -0.1980798  -0.02215572  0.28196502]
[ 0.00995592 -0.1980798  -0.02215572  0.28196502]
[ 0.00599432 -0.00264894 -0.01651641 -0.01762255]
[ 0.00599432 -0.00264894 -0.01651641 -0.01762255]
[ 0.00594135  0.19270594 -0.01686887 -0.3154705 ]
[ 0.00594135  0.19270594 -0.01686887 -0.3154705 ]
[ 0.00979546  0.38806406 -0.02317828 -0.6134252 ]
[ 0.00979546  0.38806406 -0.02317828 -0.6134252 ]
[ 0.01755675  0.19327354 -0.03544678 -0.32813174]
[ 0.01755675  0.19327354 -0.03544678 -0.32813174]
[ 0.02142222 -0.00132631 -0.04200941 -0.04683447]
[ 0.02142222 -0.00132631 -0.04200941 -0.04683447]
[ 0.02139569  0.19437206 -0.0429461  -0.3524702 ]
[ 0.02139569  0.19437206 -0.0429461  -0.3524702 ]
[ 0.02528313 -0.00011371 -0.04999551 -0.07363278]


[ 0.05753515  1.0345566  -0.19014542 -1.7954098 ]
[ 0.07822628  0.8420053  -0.22605361 -1.5673548 ]
[-0.04683615  0.0371341   0.04338782  0.00760126]
[-0.04609346  0.23160782  0.04353984 -0.2710828 ]
[-0.04609346  0.23160782  0.04353984 -0.2710828 ]
[-0.0414613   0.03589248  0.03811819  0.03500868]
[-0.0414613   0.03589248  0.03811819  0.03500868]
[-0.04074346  0.23044766  0.03881836 -0.24540807]
[-0.04074346  0.23044766  0.03881836 -0.24540807]
[-0.0361345   0.0347934   0.0339102   0.05926197]
[-0.0361345   0.0347934   0.0339102   0.05926197]
[-0.03543863 -0.16079794  0.03509544  0.36244798]
[-0.03543863 -0.16079794  0.03509544  0.36244798]
[-0.03865459  0.03380808  0.0423444   0.08103467]
[-0.03865459  0.03380808  0.0423444   0.08103467]
[-0.03797843 -0.1618945   0.04396509  0.38677093]
[-0.03797843 -0.1618945   0.04396509  0.38677093]
[-0.04121632 -0.35761207  0.05170051  0.6929852 ]
[-0.04121632 -0.35761207  0.05170051  0.6929852 ]
[-0.04836856 -0.5534117   0.06556021  1.0014853 ]


[ 0.01443104 -0.16198681  0.03036976  0.30174032]
[ 0.0111913  -0.35752815  0.03640456  0.6038444 ]
[ 0.0111913  -0.35752815  0.03640456  0.6038444 ]
[ 0.00404074 -0.16293375  0.04848145  0.32284677]
[ 0.00404074 -0.16293375  0.04848145  0.32284677]
[0.00078206 0.03146552 0.05493839 0.04583846]
[0.00078206 0.03146552 0.05493839 0.04583846]
[ 0.00141138  0.2257584   0.05585515 -0.2290174 ]
[ 0.00141138  0.2257584   0.05585515 -0.2290174 ]
[ 0.00592654  0.42003953  0.05127481 -0.50357175]
[ 0.00592654  0.42003953  0.05127481 -0.50357175]
[ 0.01432733  0.22423382  0.04120337 -0.195181  ]
[ 0.01432733  0.22423382  0.04120337 -0.195181  ]
[0.01881201 0.02854745 0.03729975 0.11020979]
[0.01881201 0.02854745 0.03729975 0.11020979]
[ 0.01938296 -0.1670886   0.03950395  0.41442332]
[ 0.01938296 -0.1670886   0.03950395  0.41442332]
[ 0.01604119 -0.36274755  0.04779242  0.71929395]
[ 0.01604119 -0.36274755  0.04779242  0.71929395]
[ 0.00878624 -0.5584971   0.0621783   1.0266285 ]
[ 0.00878624 -0.

[-0.01444163 -0.02870109  0.04159865 -0.00856739]
[-0.01501565 -0.22439417  0.0414273   0.29694477]
[-0.01501565 -0.22439417  0.0414273   0.29694477]
[-0.01950353 -0.42008147  0.0473662   0.6024    ]
[-0.01950353 -0.42008147  0.0473662   0.6024    ]
[-0.02790516 -0.22565292  0.0594142   0.32500473]
[-0.02790516 -0.22565292  0.0594142   0.32500473]
[-0.03241822 -0.03142504  0.0659143   0.05163449]
[-0.03241822 -0.03142504  0.0659143   0.05163449]
[-0.03304672 -0.22742717  0.06694698  0.36436403]
[-0.03304672 -0.22742717  0.06694698  0.36436403]
[-0.03759526 -0.4234335   0.07423426  0.67738265]
[-0.03759526 -0.4234335   0.07423426  0.67738265]
[-0.04606393 -0.6195041   0.08778191  0.9924833 ]
[-0.04606393 -0.6195041   0.08778191  0.9924833 ]
[-0.05845401 -0.42565936  0.10763158  0.7286106 ]
[-0.05845401 -0.42565936  0.10763158  0.7286106 ]
[-0.0669672  -0.23217684  0.1222038   0.47164884]
[-0.0669672  -0.23217684  0.1222038   0.47164884]
[-0.07161074 -0.03897369  0.13163677  0.21984391]


[-0.04870549  0.5425871  -0.05680177 -0.9898565 ]
[-0.03785375  0.73842144 -0.07659891 -1.2998251 ]
[-0.03785375  0.73842144 -0.07659891 -1.2998251 ]
[-0.02308532  0.9344275  -0.1025954  -1.61547   ]
[-0.02308532  0.9344275  -0.1025954  -1.61547   ]
[-0.00439677  0.7406546  -0.1349048  -1.3564492 ]
[-0.00439677  0.7406546  -0.1349048  -1.3564492 ]
[ 0.01041632  0.5474583  -0.16203378 -1.1088296 ]
[ 0.01041632  0.5474583  -0.16203378 -1.1088296 ]
[ 0.02136549  0.74429524 -0.18421037 -1.4476472 ]
[ 0.02136549  0.74429524 -0.18421037 -1.4476472 ]
[ 0.03625139  0.9411419  -0.21316333 -1.7917737 ]
[-0.01762166  0.03542467  0.02541851 -0.03451213]
[-0.01691316 -0.16005239  0.02472827  0.26608098]
[-0.01691316 -0.16005239  0.02472827  0.26608098]
[-0.02011421  0.03470806  0.03004989 -0.01870101]
[-0.02011421  0.03470806  0.03004989 -0.01870101]
[-0.01942005  0.22938646  0.02967587 -0.30175343]
[-0.01942005  0.22938646  0.02967587 -0.30175343]
[-0.01483232  0.42407316  0.0236408  -0.5849315 ]


[ 0.03955174 -0.5083106  -0.20844972 -0.05400437]
[ 0.02938553 -0.69992906 -0.2095298   0.16635998]
[-0.03929907 -0.03379386 -0.01254092  0.01921702]
[-0.03997495  0.16150567 -0.01215658 -0.27739617]
[-0.03997495  0.16150567 -0.01215658 -0.27739617]
[-0.03674483  0.35679892 -0.0177045  -0.57388836]
[-0.03674483  0.35679892 -0.0177045  -0.57388836]
[-0.02960886  0.1619296  -0.02918227 -0.28683504]
[-0.02960886  0.1619296  -0.02918227 -0.28683504]
[-0.02637026  0.3574553  -0.03491897 -0.5885771 ]
[-0.02637026  0.3574553  -0.03491897 -0.5885771 ]
[-0.01922116  0.16283931 -0.04669052 -0.30709514]
[-0.01922116  0.16283931 -0.04669052 -0.30709514]
[-0.01596437  0.3585944  -0.05283242 -0.6141298 ]
[-0.01596437  0.3585944  -0.05283242 -0.6141298 ]
[-0.00879248  0.55441326 -0.06511501 -0.92297375]
[-0.00879248  0.55441326 -0.06511501 -0.92297375]
[ 0.00229578  0.7503516  -0.08357449 -1.2353888 ]
[ 0.00229578  0.7503516  -0.08357449 -1.2353888 ]
[ 0.01730281  0.55639726 -0.10828226 -0.970015  ]


***

<h2 style="color:#A7BD3F;">Section 3 - Dynamic Programming algorithms for MDPs</h2>

The pseudocode for the general case of Value Iteration and Policy Iteration is given below.

For the environments with discrete state and action spaces, such as FrozenLake, it becomes shorter and easier to comprehend, see pp. 74-83 in Chapter 4 of the class text <sup>[3]</sup>. Be aware of the differences in the notation: $s$ instead of $x$, etc.

We have implemented a class `MDP`, containing methods that are common to these algorithms.

**\_model_transits_rewards** method implements experience gaining, i.e. performing random actions in order to make $\infty$-horizon reward prediction possible.

The resultant policies converge to the optimal policy in terms of the Functional norm, meaning that $\lim \limits_{k \rightarrow \infty} \|V_k - \hat{V}\| = \lim \limits_{k \rightarrow \infty} \sup \limits_{x \in \mathbb{X}} \|V_k(x) - \hat{V}(x)\| = 0$. However, real-world implementation requires practical termination criteria. Here we stop the procedure when the maximal change of the policy among all the states during one optimization loop does not exceed a certain constant $\nu$, see pseudocode for details. The value $\nu$ is a tuning parameter for both algorithms.

The initial values for Value Iteration could be set to $0$. Same for the Policy Iteration, but only for the finite state spaces: in continuous case the criteria for the feasible initialization are discussed later in the course.

<img src="algorithms.png" alt="Policy Iteration" width=75% height=75% />

These algorithms have very similar value-of-state estimation cycle. The difference is that the *value iteration* algorithm iterates over **every** action while calculating the expected reward, and selects action with the maximal value. While the policy iteration calculates expected reward for the **single** action **specified by the policy**.

Sidenote: if $\mathbb{X}$ space is continuous, $x_j$ for both algorithms should be taken from a finite grid $\mathbb{X}_0$ over $\mathbb{X}$. For $x$ that does not belong to the grid the value of the function $V$ could be calculated via different approximations: closest neighbor-based, linear, etc.

In [3]:
"""
DO NOT MODIFY
"""

class MDP:
    def __init__(self, env_name, is_slippery=False):
        self.env = gym.make(env_name, is_slippery=is_slippery)
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)
        self.gamma = 0.95
        self.nu = 0.0005
        
    def return_rewards(self):
        return 1.0 in self.rewards.values()

    def return_state_values(self):
        return tuple(self.values.items())

    def _model_transits_rewards(self, num_steps):
        """

        Description: step through the environment to model rewards and transits for all states. Also called "filling a buffer".

        Args:
            * num_steps - num steps to take through env. Should be sufficient to stochastically achieve goal of 1.
        """
        current_state = self.env.reset()

        print("Modeling rewards and transition probabilities ...")
    
        for i in tqdm(range(num_steps)):
            # sample random action
            action = self.env.action_space.sample()

            # take step
            new_state, reward, is_done, _ = self.env.step(action)
            
            # assign rewards for new state
            self.rewards[(current_state, action, new_state)] = reward

            # log transit from state to new state
            self.transits[(current_state, action)][new_state] += 1
            
            if is_done:
                current_state = self.env.reset()
            else:
                current_state = new_state

    def _get_action_value(self, current_state, action):
        """ 

        Description: Get the value of action

        Args:
            * State
            * Action

        Returns:
            * Value of current state

        """
        next_state_counts = self.transits[(current_state, action)]
        total_transits = sum(next_state_counts.values())
        action_value = 0.0
        
        for next_state, n_transits in next_state_counts.items():
            reward = self.rewards[(current_state, action, next_state)]
            transit_prob = (n_transits / total_transits)
            action_value += transit_prob * (reward + self.gamma * self.values[next_state])
        
        return action_value

    def _get_best_action(self, state):
        """ 

        Description: get best action for current state

        Args:
            * state

        Returns:
            * best action for current state

        """
        action_values = {}

        for action in range(self.env.action_space.n):
            action_value = self._get_action_value(state, action)
            action_values[action] = action_value

        best_action_value = max(action_values.values())
        best_action = max(action_values, key=action_values.get)
        
        return best_action

***

<h2 style="color:#A7BD3F;">Section 4: Problems</h2>

### <font color="blue">Problem 1.1 - Value Iteration</font>

Implement Value Iteration with the FrozenLake environment.

Guidance/hints:
* Read Chapter 4 from the class text
* Make sure you understand the value iteration algorithm
* Test taking steps through the FrozenLake environment by making your own script and executing it in new code cells
* Explore the MDP class above
* To complete the task, you'll need to make sure that your algorithm is properly calculating the state values. Call the `return_state_values` method to check the value of states. The value of the final state (aka the goal state) is always 0.

In [4]:
"""
ADD YOUR CODE BETWEEN THE COMMENTS BELOW
"""
class ValueIteration(MDP):
    def __init__(self, env_name, is_slippery=False):
        print('ValueIteration.__init__(self) called')
        super().__init__(env_name, is_slippery=is_slippery)
        
    def _value_iteration(self):
        """ 

        Description: Perform Value Iteration

        """
        # print("Performing value iteration ...")

        delta = 0
        iter_count = 1

        while True:
            ### YOUR SOLUTION BELOW
            delta = 0                

            for state in range(self.env.nS):
                saved_value = self.values[state]
                
#                 V_a = [0] * self.env.nA
#                 for action in range(self.env.nA):
#                     for prob, next_state, reward, _ in self.env.P[state][action]:
#                         V_a[action] += prob * (reward + self.gamma * self.values[next_state])
#                 best_action = np.argmax(V_a)
                
                best_action = self._get_best_action(state)
                self.values[state] = self._get_action_value(state, best_action)
                delta = max(delta, abs(saved_value - self.values[state]))

            ### YOUR SOLUTION ABOVE
            
            iter_count += 1

            if delta < self.nu:
                break                   
                
    def _run_episode(self, render=True):
        """

        Description: perform an episode on the environment

        Args
            * Render - render env to screen?

        """

        clear_output()
        episode_reward = 0.0
        current_state = self.env.reset()

        if render:
            self.env.render()
        
        while True:
            action = self._get_best_action(current_state)
            new_state, step_reward, is_done, _ = self.env.step(action)
            
            self.rewards[(current_state, action, new_state)] = step_reward
            self.transits[(current_state, action)][new_state] += 1
            
            episode_reward += step_reward
            
            if render:
                self.env.render()

            if is_done:
                self.env.reset()
                break
            
            current_state = new_state

        print(f"...Episode completed.")

        return episode_reward
        
    def run_simulation(self, num_steps = 1000, render=True):
        """ Run training simulation """
        try:
            ### YOUR SOLUTION BELOW
            
            self._model_transits_rewards(num_steps)
            self._value_iteration()
            
            ### YOUR SOLUTION ABOVE
            
            episode_reward = self._run_episode(render=render)
            if episode_reward > 0.85:
                print(f"Environment solved.")
            else:
                clear_output()
                print(f"Failed to solve environment.")

        except KeyboardInterrupt:
            print(f"...Cancelling...")
            
        except NotImplementedError:
            print(f"Your solution is incomplete")

### Run simulation

Execute the cell below. If your code is correct, you will see the environment render the agent taking steps. The final cell will be 'G'. 

**Note**: Running the simulation below with a large enough `num_steps` hyperparameter is vital to the value iteration algorithm working successfully. This is because the modeling process, which is stochastic (via `_model_transits_rewards`), requires sufficient iterations to reach the goal-state and acquire the reward in the environment, as well as to make enough transits to all possible states to make value iteration numerically stable.

In [5]:
# DO NOT MODIFY

start_time = time.time()

agent1 = ValueIteration("FrozenLake-v1")
agent1.run_simulation(num_steps = 3000)

end_time = time.time()
print(f"Duration of execution: {end_time-start_time:.5f} seconds")


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
...Episode completed.
Environment solved.
Duration of execution: 0.05303 seconds


In [6]:
#agent1.return_state_values()
agent1.env.observation_space.n
agent1.values

defaultdict(float,
            {0: 0.7737809374999999,
             4: 0.8145062499999999,
             1: 0.8145062499999999,
             5: 0.0,
             2: 0.8573749999999999,
             6: 0.9025,
             3: 0.8145062499999999,
             7: 0.0,
             8: 0.8573749999999999,
             10: 0.95,
             12: 0.0,
             9: 0.9025,
             13: 0.95,
             14: 1.0,
             11: 0.0,
             15: 0.0})

### State values?

In a 4x4 grid of FrozenLake, there are 16 states. Now let's look at their values:

In [7]:
state_values = agent1.return_state_values()
state_values

((0, 0.7737809374999999),
 (4, 0.8145062499999999),
 (1, 0.8145062499999999),
 (5, 0.0),
 (2, 0.8573749999999999),
 (6, 0.9025),
 (3, 0.8145062499999999),
 (7, 0.0),
 (8, 0.8573749999999999),
 (10, 0.95),
 (12, 0.0),
 (9, 0.9025),
 (13, 0.95),
 (14, 1.0),
 (11, 0.0),
 (15, 0.0))

In [8]:
rewards = agent1.return_rewards()
rewards

True

### <font color="orange">Auto-grading</font>
Run this cell to track your answers and to save your answer for problem 1.1. Make sure you defined the necessary variable above to avoid a `NameError` below.

In [9]:
### GRADING DO NOT MODIFY
from grading_utilities import AnswerTracker
asgn1_answers = AnswerTracker()
rewards_values = agent1.return_rewards()
asgn1_answers.record('problem_1-1', {'state_values': state_values, 'rewards': rewards_values})

### <font color="blue">Problem 1.2 - Policy Iteration</font>

Implement Policy Iteration on the FrozenLake environment.

In [10]:
class PolicyIteration(MDP):
    def __init__(self, env_name, is_slippery=False):
        super().__init__(env_name, is_slippery=is_slippery)
        self.policy = collections.defaultdict(int)
        
    def return_policy(self):
        return tuple(self.policy.items())
        
    def _policy_iteration(self):
        """ 
        
        Description: Perform policy iteration. Consists of 2 parts: policy evaluation and policy improvement. See Sutton & Barto, RL: An Introduction, page 80.
        
        """

        #-------------------#
        # policy evaluation #
        #-------------------#

        delta = 0
        iter_count = 1

        while True:
            ### YOUR SOLUTION BELOW
            delta = 0
            for state in range(self.env.nS):
                saved_value = self.values[state]

#                 V_a = [0] * self.env.nA
#                 policy = [self.policy[state, action] for action in range(self.env.nA)]
#                 policy_action = np.argmax(policy)
#                 for prob, next_state, reward, _ in self.env.P[state][policy_action]:
#                     V_a[policy_action] += prob * (reward + self.gamma * self.values[next_state])
#                 self.values[state] = np.sum(V_a)
                    
                best_action = self.policy[state]
                new_value = self._get_action_value(state, best_action)

                self.values[state] = new_value
                delta = max(delta, abs(new_value - saved_value))
            
            #print('iteration {}, delta = {}'.format(iter_cout, delta)})

            ### YOUR SOLUTION ABOVE
            
            iter_count += 1

            if delta < self.nu:
                break

        #--------------------#
        # policy improvement #
        #--------------------#
        
        policy_stable = np.zeros((self.env.observation_space.n), dtype=bool) 

        ### YOUR SOLUTION BELOW
    
        for state in range(self.env.nS):            
            
            old_action = self.policy[state]

#             old_policy = [self.policy[state, action] for action in range(self.env.nA)]
#             old_action = np.argmax(old_policy)
#             new_policy = [0] * self.env.nA
#             for action in range(self.env.nA):
#                 self.policy[state, action] = 0
#                 for prob, next_state, reward, _ in self.env.P[state][action]:
#                     new_policy[action] += prob * (reward + self.gamma * self.values[next_state])
#             new_action = np.argmax(new_policy)
#             self.policy[state, new_action] = 1
#             if old_action == new_action:
#                 policy_stable[state] = True     

            new_action = self._get_best_action(state)
            self.policy[state] = new_action
            
            # print('old action = {}, new action = {}'.format(old_action, new_action))
            
            if old_action == new_action:
                policy_stable[state] = True     
        
        ### YOUR SOLUTION ABOVE

        return policy_stable
    
    def _run_episode(self, render=True):
        """

        Description: runs an episode on the environment after policy iteration.

        Args:
            * Render - render env to screen?

        """
        clear_output()
        episode_reward = 0.0
        current_state = self.env.reset()
        
        if render:
            self.env.render()
        
        while True:
            action = self.policy[current_state]
            new_state, step_reward, is_done, _ = self.env.step(action)
            
            self.rewards[(current_state, action, new_state)] = step_reward
            self.transits[(current_state, action)][new_state] += 1
            
            episode_reward += step_reward
            
            if render:
                self.env.render()

            if is_done:
                self.env.reset()
                break
            
            current_state = new_state
            
        print(f"...Episode completed.")

        return episode_reward
        
    def run_simulation(self, num_steps = 2000, render=True):
        """ Run training simulation """
        try:
            self._model_transits_rewards(num_steps)

            while True:
                policy_stable = self._policy_iteration()

                if policy_stable.all() == True:
                    break

            episode_reward = self._run_episode(render=render)
            
            if episode_reward > 0.85:
                print(f"Environment solved.")
            else:
                clear_output()
                print(f"Failed to solve environment.")
        
        except KeyboardInterrupt:
            print(f"...Cancelling...")
            
        except NotImplementedError:
            print(f"Your solution is incomplete")

### Run simulation

As discussed above, beware of the num_steps hyperparam.

In [11]:
start_time = time.time()

agent2 = PolicyIteration("FrozenLake-v1")
agent2.run_simulation(num_steps = 3000)

end_time = time.time()
print(f"Duration of execution: {end_time-start_time:.5f} seconds")


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
...Episode completed.
Environment solved.
Duration of execution: 0.05203 seconds


### <font color="orange">Auto-grading</font>
Run this cell to track your answers and to save your answer for problem 1.2. Make sure you defined the necessary variable above to avoid a `NameError` below.

In [12]:
state_values = agent2.return_state_values()
policy_values = agent2.return_policy()

### GRADING DO NOT MODIFY
asgn1_answers.record('problem_1-2', {'state_values': state_values, 'policy_values': policy_values})

### <font color="blue">Problem 1.3 - FrozenLake8x8</font>

Let's try the 8x8 version of FrozenLake and compare which algorithm is faster.

In [13]:
start_time = time.time()

# value iteration
agent3 = ValueIteration("FrozenLake8x8-v1")
agent3.run_simulation(render = False, num_steps = 50000)

end_time = time.time()
print(f"VI: Duration of execution: {end_time-start_time:.5f} seconds")

...Episode completed.
Environment solved.
VI: Duration of execution: 1.11003 seconds


In [14]:
%%timeit
agent3._value_iteration()

522 µs ± 674 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
start_time = time.time()

# policy iteration
agent4 = PolicyIteration("FrozenLake8x8-v1")
agent4.run_simulation(render = False, num_steps = 50000)

end_time = time.time()
print(f"PI: Duration of execution: {end_time-start_time:.5f} seconds")

...Episode completed.
Environment solved.
PI: Duration of execution: 1.00198 seconds


In [16]:
%%timeit
agent4._policy_iteration()

333 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Which algorithm is faster?

Which algorithm do you think is faster, on average, policy iteration (PI) or value iteration (VI)? Why?

In [17]:
"""
Change None to 'PI' or 'VI' below
"""
### YOUR ANSWER BELOW
problem_13_answer = 'PI'
### YOUR ANSWER ABOVE

Seems that policy iteration is s big more complex algorthim, then value iteration, however it requires less iterations and converges faster. Policy iteration step consists of 2 phases: evaluation and improvement whereas value iteration loops over all actions to find the best one, so each step of value iteration is faster that policy iteration step, but PI steps are more effective as PI takes less iterations.

### <font color="orange">Auto-grading</font>
Run this cell to track your answers and to save your answer for problem 1.3. Make sure you defined the necessary variable above to avoid a `NameError` below.

In [18]:
### GRADING DO NOT MODIFY
asgn1_answers.record('problem_1-3', problem_13_answer)

### <font color="orange">Auto-grading: Submit your answers</font>
Enter your first and last name in the cell below and then run it to save your answers for this lab to a JSON file. The file is saved to the same directory as this notebook. After the file is created, upload the JSON file to the assignment page on Canvas.

In [19]:
asgn1_answers.print_answers()

{'problem_1-1': {'state_values': ((0, 0.7737809374999999), (4, 0.8145062499999999), (1, 0.8145062499999999), (5, 0.0), (2, 0.8573749999999999), (6, 0.9025), (3, 0.8145062499999999), (7, 0.0), (8, 0.8573749999999999), (10, 0.95), (12, 0.0), (9, 0.9025), (13, 0.95), (14, 1.0), (11, 0.0), (15, 0.0)), 'rewards': True}, 'problem_1-2': {'state_values': ((0, 0.7737809374999999), (1, 0.8145062499999999), (2, 0.8573749999999999), (3, 0.8145062499999999), (4, 0.8145062499999999), (5, 0.0), (6, 0.9025), (7, 0.0), (8, 0.8573749999999999), (9, 0.9025), (10, 0.95), (11, 0.0), (12, 0.0), (13, 0.95), (14, 1.0), (15, 0.0)), 'policy_values': ((0, 1), (1, 2), (2, 1), (3, 0), (4, 1), (5, 0), (6, 1), (7, 0), (8, 2), (9, 1), (10, 1), (11, 0), (12, 0), (13, 2), (14, 2), (15, 0))}, 'problem_1-3': 'PI'}


In [20]:
assignment_name = "asgn_1"
first_name = "Albina"
last_name = "Klepach"

asgn1_answers.save_to_json(assignment_name, first_name, last_name)

## Questions?

Reach out to Ilya Osokin (@elijahmipt) on Telegram.

## Sources

***

<sup>[1]</sup> Ng, A. Stanford University, CS229 Notes: Reinforcement Learning and Control.

<sup>[2]</sup> Barnabás Póczos, Carnegie Mellon, Introduction To Machine Learning: Reinforcement Learning (Course).

<sup>[3]</sup> **Sutton, R. S., Barto, A. G. (2018 ). Reinforcement Learning: An Introduction. The MIT Press.** 

<sup>[4]</sup> OpenAI: Spinning Up. Retrieved from https://spinningup.openai.com/en/latest/spinningup/rl_intro.html